Ok so. We're going to go through all of this again<br>
First test, importing libraries<br>
Then embedding function<br>
Then getting those embeddings<br>
We will have to figure a way to send those to the DB<br>
but that's ok<br>
Then averaging posts user likes<br>
And sending those back somehow<br>
It sounds so easy broken into those steps.

In [22]:
# Imports
import pandas as pd
from sentence_transformers import SentenceTransformer
import sqlite3
from sklearn.metrics.pairwise import cosine_similarity
import random
import re
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import json

Ok for some reason that only took a second.<br>
Now we need to get the posts and run the embeddings on them<br>
Actually first we need to make a place to put the vectors<br>

In [15]:
# This isn't going to be used for now but we'll include it anyway.
def camel_case_split(str):
    return re.sub(r'([a-z])([A-Z])',
                      r'\1 \2', str)

In [3]:
# Ok so we don't actually need a place to put the vectors.
# Because SQLite doesn't do that
# We will need a way to separate tags in the actual post body
# But that's a problem for future Ace when they have access to that specific data.

def call_for_data(user_id):
    con = sqlite3.connect("/home/acequantum/playtime/dummy_data/1_dummy_data.db")

    df_all_posts = pd.read_sql_query("SELECT * FROM posts;", con)

    liked_posts_query = \
    f"SELECT posts.tags \
    FROM likes \
    JOIN posts on likes.post_id = posts.id \
    WHERE likes.user_id = {user_id}"

    user_query = f"SELECT * FROM users WHERE id={user_id}"

    df_liked_posts = pd.read_sql_query(liked_posts_query, con)

    df_user = pd.read_sql_query(user_query, con)

    return df_all_posts, df_user, df_liked_posts

In [12]:
random.seed(10)
user = random.randint(0,500)

df_all_posts, df_user, df_liked_posts = call_for_data(user)

print(f"all posts: {df_all_posts}")
print(f"user info: {df_user}")
print(f"user name: {df_user['name']}")
print(f"user liked posts: {df_liked_posts.head}")

all posts:           id  ...                                               text
0          1  ...  Really describe particularly whose opportunity...
1          2  ...  Analysis father board everybody. Car begin evi...
2          3  ...  Per assume community dog happy really stop. Ch...
3          4  ...  By song new know. Particularly third all inves...
4          5  ...  Three writer policy building. Offer the capita...
...      ...  ...                                                ...
26259  26260  ...  Up go also hotel one sometimes kitchen. Box va...
26260  26261  ...  Push each difficult someone. Star increase usu...
26261  26262  ...  Make well each near. Finish in market official...
26262  26263  ...  Example ask sort security husband fast and mis...
26263  26264  ...  Yes eye weight from when political. Building p...

[26264 rows x 5 columns]
user info:     id  ...                                               bios
0  292  ...  Significant suggest respond ask born. Speak re..

In [23]:
# Ignore previous comment about not needing to tokenize those tags.
# We're going to do that
# Then embed the whole of them total.
# And also embed the body of the post I guess.
# We also need to include goods but we don't have access to goods.
# So I guess we'll just end up concatinating goods?

df_all_posts['tags'] = df_all_posts['tags'].apply(camel_case_split)
print(df_all_posts['tags'].head())
df_all_posts['split_tags'] = df_all_posts['tags'].str.split(', ')
print(df_all_posts['split_tags'].head())


0    manga Anime Merch, diy Tools, mechanical Keybo...
1    collectible Pins, pet Sitting Dog Walking, vin...
2                             custom Toys, baked Goods
3    clay Modeling Kits, coding Lessons, tattoo Des...
4                 leather Goods, bike Repairs, watches
Name: tags, dtype: object
0    [manga Anime Merch, diy Tools, mechanical Keyb...
1    [collectible Pins, pet Sitting Dog Walking, vi...
2                           [custom Toys, baked Goods]
3    [clay Modeling Kits, coding Lessons, tattoo De...
4               [leather Goods, bike Repairs, watches]
Name: split_tags, dtype: object


In [31]:
# Generation of embeddings

model = SentenceTransformer('all-MiniLM-L6-v2')

def generated_weighted_embeddings(post):
    clean_tags = preprocess_tags(post["tags"])

    tags = post['tags']
    # profile = post['user_id']
    # goods when we get there
    # goods = post['goods']
    text = post['text']

    tag_embedding = model.encode(" ".join(clean_tags)) if clean_tags else np.zeros(384)
    # This is where we would put goods embeddings
    # Idk if we want to weigh it by 2.0 but that's something we can play with,
    # goods_embedding = model.encode(goods) * 2.0
    # profile_embedding = model.encode(profile)if profile else np.zeros(384)
    text_embedding = model.encode(text) if text else np.zeros(384)

    combined = np.concatenate([
        tag_embedding,
        # profile_embedding * 0.5,
        text_embedding * 0.3
    ])

    return combined

In [24]:
def preprocess_tags(tags):
    if isinstance(tags, str):
        tags = camel_case_split(tags)
        return [tag.strip() for tag in tags.split(',') if tag.strip()]
    elif isinstance(tags, list):
        return [camel_case_split(tag) for tag in tags]
    return []

In [34]:
# This is where we insert embeddings into the posts
# This will change when we move things to postgres.

con = sqlite3.connect("/home/acequantum/playtime/dummy_data/1_dummy_data.db")
cursor = con.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS post_embeddings (
        post_id INTEGER PRIMARY KEY,
        embedding BLOB)""")

cursor.execute("SELECT id, tags, user_id, text FROM posts")
posts = [
    {"id": row[0], "tags": row[1], "user_id": row[2], "text": row[3]}
    for row in cursor.fetchall()
]

# Here's the precomputing
for post in posts:
    embedding = generated_weighted_embeddings(post)
    print(f"Generated embedding for post {post['id']}")
    cursor.execute(
        "INSERT OR REPLACE INTO post_embeddings (post_id, embedding) \
        VALUES (?, ?)",
        (post["id"], json.dumps(embedding.tolist()))
    )
con.commit()

Generated embedding for post 1
Generated embedding for post 2
Generated embedding for post 3
Generated embedding for post 4
Generated embedding for post 5
Generated embedding for post 6
Generated embedding for post 7
Generated embedding for post 8
Generated embedding for post 9
Generated embedding for post 10
Generated embedding for post 11
Generated embedding for post 12
Generated embedding for post 13
Generated embedding for post 14
Generated embedding for post 15
Generated embedding for post 16
Generated embedding for post 17
Generated embedding for post 18
Generated embedding for post 19
Generated embedding for post 20
Generated embedding for post 21
Generated embedding for post 22
Generated embedding for post 23
Generated embedding for post 24
Generated embedding for post 25
Generated embedding for post 26
Generated embedding for post 27
Generated embedding for post 28
Generated embedding for post 29
Generated embedding for post 30
Generated embedding for post 31
Generated embeddi